# Deploying an endpoint from a model and enabling data capture with SageMaker Model Monitor

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [ ]:
%store -r model_name
model_name

In [ ]:
import boto3
client = boto3.client('sagemaker')

In [ ]:
response = client.describe_model(ModelName=model_name)
response

In [ ]:
container = response['PrimaryContainer']['Image']
container

In [ ]:
model_data = response['PrimaryContainer']['ModelDataUrl']
model_data

In [ ]:
model = sagemaker.model.Model(
    name=model_name,
    image_uri=container,
    model_data=model_data,
    role=role,
    sagemaker_session=session
)

In [ ]:
%%time

model.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.m5.large'
)

In [ ]:
model.endpoint_name

In [ ]:
from sagemaker import Predictor
predictor = Predictor(endpoint_name=model.endpoint_name)

In [ ]:
%store -r s3_bucket_name
%store -r prefix
base = f"s3://{s3_bucket_name}/{prefix}"
s3_capture_upload_path = f"{base}/model-monitor"

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

In [ ]:
data_capture_config = DataCaptureConfig(
    enable_capture = True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
    kms_key_id=None,
    capture_options=["REQUEST", "RESPONSE"],
    csv_content_types=["text/csv"],
    json_content_types=["application/json"]
)

In [ ]:
%%time

predictor.update_data_capture_config(
    data_capture_config=data_capture_config
)

In [ ]:
boto_session = boto3.Session(region_name=region)
runtime = boto_session.client(
    service_name='sagemaker-featurestore-runtime', 
    region_name=region
)

In [ ]:
feature_group_name = 'cookbook-feature-group'

record_response = runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString="950"
)

In [ ]:
record_response['Record']

In [ ]:
test_record_list = [
    record_response['Record'][1]['ValueAsString'],
    record_response['Record'][2]['ValueAsString'],
    record_response['Record'][3]['ValueAsString'],
    record_response['Record'][4]['ValueAsString'],
    record_response['Record'][5]['ValueAsString'],
    record_response['Record'][6]['ValueAsString'],
]

test_record_list

In [ ]:
csv_input = ','.join(test_record_list)
csv_input

In [ ]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [ ]:
predictor.predict(csv_input)

In [ ]:
endpoint_name = predictor.endpoint_name

%store endpoint_name
endpoint_name

In [ ]:
%store csv_input